# Web-scraping: сбор данных из баз данных и интернет-источников

*Алла Тамбовцева, НИУ ВШЭ*

## Более продвинутый парсинг с BeautifulSoup

Импортируем необходимые модули и библиотеки:

In [ ]:
import requests
import re
import pandas as pd
from bs4 import BeautifulSoup

### Часть 1: чуть-чуть про кодировки

Подключаемся к странице древнего [сайта-архива](https://filmiki.arjlover.net/filmiki/) с детскими (преимущественно, советскими) фильмами:

In [ ]:
page01 = requests.get("https://filmiki.arjlover.net/filmiki/")
soup01 = BeautifulSoup(page01.text)

Давайте найдём на этой странице таблицу, в которой сохранены названия фильмов и ссылки для скачивания.

In [ ]:
### YOUR CODE HERE ###

Проблема: текст в таблице не совсем корректно считывается, вместо букв крокозябры. Эта проблема связана с тем, что кодировка, которую функция `.get()` выбрала здесь по умолчанию, не соответствует той, которая нам нужна. Посмотрим на неё:

In [ ]:
page01.encoding

Изменим её:

In [ ]:
page01.encoding='windows-1251'

Попробуем повторить те же действия:

In [ ]:
soup01 = BeautifulSoup(page01.text)
table = soup01.find_all("table")[3]
table

Выглядит получше! Давайте вспомним, как превращать код HTML в датафрейм `pandas`:

In [ ]:
### YOUR CODE HERE ###

Минутка священного рандома – вдруг пригодится:

In [ ]:
### YOUR CODE HERE ###

### Часть 2: дополнительные опции в BeautifulSoup или обходим капчи

Итак, мы все-таки ограбим Кинопоиск! Попробуем сначала действовать как обычно:

In [ ]:
url = "https://www.kinopoisk.ru/film/44587/"
page02 = requests.get(url)

Результат нас вряд ли устроит:

In [ ]:
page02.text

Посмотрим на страницу, которая открывается при попытке сгрузить информацию через `get()`, оно того стоит:

In [ ]:
# создаем новый файл HTML, w – от write, режим записи
# декодируем, чтобы был текст на кириллице
# вписываем содержимое в файл
# закрываем файл для сохранения изменений

check = open('check.html', 'w')
check.write(page02.text.encode('cp1251').decode())
check.close()

Зайдем в инструменты разработчика и найдем опцию, которая добавляется при обработке запроса к странице, когда мы открываем ее в «ручном» режиме:

In [ ]:
headers = {'user-agent':
           'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36'}

Учтем эту опцию при загрузке страницы:

In [ ]:
page02 = requests.get("https://www.kinopoisk.ru/film/44587/", headers = headers)
soup02 = BeautifulSoup(page02.text)

Ура! Исходный код страницы получен, можно его парсить.

### Часть 3: продолжаем поиски

Попытаемся найти описание фильма, которое начинается со слов *Художник-безумец*:

In [ ]:
soup02.find("meta", {"name" : "description"}) # здорово, но не то

А если найдем все тэги `<meta>` и проверим?

In [ ]:
soup02.find_all("meta") # нет такого...

А есть ли вообще это описание в исходном коде страницы в `soup02`? Давайте превратим этот объект в обычную строку и поищем:

In [ ]:
text = str(soup02)
text.find("Художник-безумец")

Вообще-то есть, нам вернулся индекс начала такой подстроки. Выберем фрагмент текста побольше и пораньше:

In [ ]:
text[122000:]

Воспользуемся регулярными выражениями:

In [ ]:
description = re.findall('"shortDescription":"(.+?)",', text)[0]
print(description)

Теперь найдем раздел, который содержит большую часть информации о фильме (заголовок *О фильме*) и сохраним фрагмент кода в переменную `tab`:

In [ ]:
### YOUR CODE HERE ###

Найдем все элементы внутри, извлечем из них текст и создадим словарь с информацией о фильме:

In [ ]:
### YOUR CODE HERE ###

Найдем ссылку, которая ведет нас к подробной информации о рейтинге фильма, и сохраним ее в переменную `link`:

In [ ]:
### YOUR CODE HERE ###

Сделаем ссылку полной, перейдем по ней:

In [ ]:
link_full = "https://www.kinopoisk.ru" + link
print(link_full)

Начинаем все сначала:

In [ ]:
page03 = requests.get(link_full, headers = headers)
soup03 = BeautifulSoup(page03.text)

Найдем и сохраним рейтинг фильма и количество оценок:

In [ ]:
### YOUR CODE HERE ###

### Часть 4: забираем данные из json-файла и воспроизводим графики

А теперь - самое интересное! Заберем данные, которые используются для построения графиков на этой странице, чтобы самим строить графики не хуже (а то и лучше). Остановимся пока на тех данных, которые легли в основу круговой диаграммы.

Данные хранятся в json-файле, ссылка на который доступна внутри кода JavaScript в исходном коде страницы. Для универсальности и надежности превратим `soup03` в обычную строку и вновь воспользуемся регулярными выражениями, как раньше:

In [ ]:
text03 = str(soup03)
re.findall("dataURI:(.+),", text03)

In [ ]:
json_link = re.findall("dataURI:(.+),", text03)[0].replace("'", "").strip()
print(json_link)

Переходим на страницу с этим json-файлом и забираем данные:

In [ ]:
page04 = requests.get(json_link)

In [ ]:
page04.json()

In [ ]:
data = page04.json()

Заберем значения оценок (`values`) и соответствующие им частоты (`freqs`):

In [ ]:
freqs = [d["value"] for d in data]
values = [d["title"] for d in data]

Импортируем модуль для построения графиков:

In [ ]:
from matplotlib import pyplot as plt

Вместо круговой диаграммы строим столбиковую диаграмму:

In [ ]:
plt.bar(values, freqs)

Добавляем свои цвета для столбцов – сохраняем их в виде списка `colors`:

In [ ]:
### YOUR CODE HERE ###

Обновляем график:

In [ ]:
plt.bar(values, freqs, color = colors)